## MLflow's Model Registry

In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d5680fa0423945cb9399a2de73fba750, rmse: 6.3040
run id: 469fe065b03b42daa88080ee94c9c6fc, rmse: 6.4092
run id: fee1af599f7045e8bcfc5d377537d13d, rmse: 6.4256
run id: f6b0b089a0cb4bad8621cc97fb477688, rmse: 6.4330
run id: 64a4b1e88a8e476f84807ee3252b24e2, rmse: 6.4379


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
run_id = "d5680fa0423945cb9399a2de73fba750"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/26 19:08:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653592121932, current_stage='None', description=None, last_updated_timestamp=1653592121932, name='nyc-taxi-regressor', run_id='d5680fa0423945cb9399a2de73fba750', run_link=None, source='./mlruns/1/d5680fa0423945cb9399a2de73fba750/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [11]:
model_uri

'runs:/d5680fa0423945cb9399a2de73fba750/model'

In [17]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

[<ModelVersion: creation_timestamp=1653592121932, current_stage='None', description=None, last_updated_timestamp=1653592121932, name='nyc-taxi-regressor', run_id='d5680fa0423945cb9399a2de73fba750', run_link=None, source='./mlruns/1/d5680fa0423945cb9399a2de73fba750/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>]

In [25]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None
version: 2, stage: Staging


In [28]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653592121932, current_stage='Staging', description=None, last_updated_timestamp=1653593360576, name='nyc-taxi-regressor', run_id='d5680fa0423945cb9399a2de73fba750', run_link=None, source='./mlruns/1/d5680fa0423945cb9399a2de73fba750/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [29]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653592121932, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2022-05-26', last_updated_timestamp=1653593365883, name='nyc-taxi-regressor', run_id='d5680fa0423945cb9399a2de73fba750', run_link=None, source='./mlruns/1/d5680fa0423945cb9399a2de73fba750/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [53]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    print(model.metadata)
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [46]:
df = read_dataframe("~/data/green_tripdata_2021-03.parquet")

In [47]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/ubuntu/notebooks/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [48]:
import pickle

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [49]:
X_test = preprocess(df, dv)

In [50]:
target = "duration"
y_test = df[target].values

In [54]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[19:43:13] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
artifact_path: models_mlflow
flavors:
  python_function:
    data: model.xgb
    env: conda.yaml
    loader_module: mlflow.xgboost
    python_version: 3.9.12
  xgboost:
    code: null
    data: model.xgb
    model_class: xgboost.core.Booster
    xgb_version: 1.6.1
mlflow_version: 1.26.0
model_uuid: c7451a650ab0405db7fecf9a5883c671
run_id: d5680fa0423945cb9399a2de73fba750
utc_time_created: '2022-05-26 04:51:40.482424'

CPU times: user 25.2 s, sys: 24 ms, total: 25.3 s
Wall time: 6.73 s


{'rmse': 6.250963441118867}

In [55]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[19:43:20] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
artifact_path: model
flavors:
  python_function:
    data: model.xgb
    env: conda.yaml
    loader_module: mlflow.xgboost
    python_version: 3.9.12
  xgboost:
    code: null
    data: model.xgb
    model_class: xgboost.core.Booster
    xgb_version: 1.6.1
mlflow_version: 1.26.0
model_uuid: 39974ed5cadc42078e9ad1422469b950
run_id: a7b7df7234d84b858c29a5e668063ae4
signature:
  inputs: '[{"type": "tensor", "tensor-spec": {"dtype": "float64", "shape": [-1, 13221]}}]'
  outputs: '[{"type": "tensor", "tensor-spec": {"dtype": "float32", "shape": [-1]}}]'
utc_time_created: '2022-05-26 05:04:41.825052'

CPU times: user 27.1 s, sys: 19.9 ms, total: 27.1 s
Wall time: 7.11 s


{'rmse': 6.250963441118867}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
     archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>